In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('data/Tweets.csv')
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [2]:
reviews = np.array(data['text'])[:14000]
labels = np.array(data['airline_sentiment'])[:14000]

In [6]:
data['airline_sentiment'].loc[14639]

'neutral'

In [7]:
from collections import Counter

Counter(labels)

Counter({'negative': 8679, 'neutral': 3017, 'positive': 2304})

In [8]:
punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'


all_reviews = 'separator'.join(reviews)
all_reviews = all_reviews.lower()
all_text = ''.join([c for c in all_reviews if c not in punctuation])


reviews_split = all_text.split('separator')
all_text = ' '.join(reviews_split)


words = all_text.split()

In [9]:
# get rid of web address, twitter id, and digit
new_reviews = []
for review in reviews_split:
    review = review.split()
    new_text = []
    for word in review:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)
    new_reviews.append(new_text)

In [10]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}


reviews_ints = []
for review in new_reviews:
    reviews_ints.append([vocab_to_int[word] for word in review])

In [11]:
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()


print('Tokenized review: \n', reviews_ints[:1])

Unique words:  16727

Tokenized review: 
 [[57, 213]]


In [12]:
encoded_labels = []
for label in labels:
    if label == 'neutral':
        encoded_labels.append(1)
    elif label == 'negative':
        encoded_labels.append(0)
    else:
        encoded_labels.append(1)

encoded_labels = np.asarray(encoded_labels)

In [ ]:
def pad_features(reviews_ints, seq_length):

    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

   
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [ ]:
seq_length = 30

features = pad_features(reviews_ints, seq_length=seq_length)


assert len(features)==len(reviews_ints), 
assert len(features[0])==seq_length, 


print(features[:10,:10])

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 446]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]


In [15]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]


print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(11200, 30) 
Validation set: 	(1400, 30) 
Test set: 		(1400, 30)


In [16]:
import torch
from torch.utils.data import TensorDataset, DataLoader


train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))


batch_size = 50


train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [18]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [ ]:
import torch.nn as nn

class SentimentRNN(nn.Module):
 
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
       
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
       
        self.dropout = nn.Dropout(0.3)
        
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
       
        batch_size = x.size(0)

       
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        sig_out = self.sig(out)
        
       
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        
        
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [ ]:

vocab_size = len(vocab_to_int)+1
output_size = 1
embedding_dim = 200
hidden_dim = 128
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(16728, 200)
  (lstm): LSTM(200, 128, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [ ]:
# loss
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
# training params

epochs = 10 

counter = 0
print_every = 100
clip=5 
if(train_on_gpu):
    net.cuda()

net.train()

for e in range(epochs):
    
    h = net.init_hidden(batch_size)

    
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

       
        h = tuple([each.data for each in h])

        
        net.zero_grad()

        
        output, h = net(inputs, h)

       
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        
        if counter % print_every == 0:
           
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/10... Step: 100... Loss: 0.538076... Val Loss: 0.508536
Epoch: 1/10... Step: 200... Loss: 0.390379... Val Loss: 0.452866
Epoch: 2/10... Step: 300... Loss: 0.465221... Val Loss: 0.460314
Epoch: 2/10... Step: 400... Loss: 0.439125... Val Loss: 0.420238
Epoch: 3/10... Step: 500... Loss: 0.162746... Val Loss: 0.474010
Epoch: 3/10... Step: 600... Loss: 0.236936... Val Loss: 0.464497
Epoch: 4/10... Step: 700... Loss: 0.068420... Val Loss: 0.635738
Epoch: 4/10... Step: 800... Loss: 0.272876... Val Loss: 0.550906
Epoch: 5/10... Step: 900... Loss: 0.083559... Val Loss: 0.560300
Epoch: 5/10... Step: 1000... Loss: 0.154121... Val Loss: 0.611510
Epoch: 5/10... Step: 1100... Loss: 0.075962... Val Loss: 0.617386
Epoch: 6/10... Step: 1200... Loss: 0.021026... Val Loss: 0.787925
Epoch: 6/10... Step: 1300... Loss: 0.021559... Val Loss: 0.696904
Epoch: 7/10... Step: 1400... Loss: 0.021226... Val Loss: 0.822984
Epoch: 7/10... Step: 1500... Loss: 0.008138... Val Loss: 0.812710
Epoch: 8/10... Step

In [ ]:


test_losses = [] 
num_correct = 0


h = net.init_hidden(batch_size)

net.eval()

for inputs, labels in test_loader:

 
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
  
    output, h = net(inputs, h)
    
   
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
   
    pred = torch.round(output.squeeze())  
    
   
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)



# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.898
Test accuracy: 0.816


In [24]:
# negative test review
test_review = "@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??"

In [ ]:
def tokenize_review(test_review):
    test_review = test_review.lower()

    test_text = ''.join([c for c in test_review if c not in punctuation])


    test_words = test_text.split()
    

    new_text = []
    for word in test_words:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)


    test_ints = []
    test_ints.append([vocab_to_int[word] for word in new_text])

    return test_ints

test_ints = tokenize_review(test_review)
print(test_ints)

[[5, 22, 11, 367, 5, 126, 11, 8, 10, 85, 335, 21, 922, 93, 194, 1550, 44, 3, 34, 125, 11, 2888]]


In [ ]:
seq_length=30
features = pad_features(test_ints, seq_length)

print(features)

[[   0    0    0    0    0    0    0    0    5   22   11  367    5  126
    11    8   10   85  335   21  922   93  194 1550   44    3   34  125
    11 2888]]


In [ ]:
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())

torch.Size([1, 30])


In [ ]:
def predict(net, test_review, sequence_length=30):
    
    net.eval()
    
   
    test_ints = tokenize_review(test_review)
    
    
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
 
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
   
    output, h = net(feature_tensor, h)

    pred = torch.round(output.squeeze()) 
  
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
  
    if(pred.item()==1):
        print("Non-negative review detected.")
    else:
        print("Negative review detected.")
        

In [ ]:
seq_length = 30 

In [ ]:
test_review_neg = "@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??"
predict(net, test_review_neg, seq_length)

Prediction value, pre-rounding: 0.008724
Negative review detected.


In [ ]:
test_review_pos = "@AmericanAir thank you we got on a different flight to Chicago."
predict(net, test_review_pos, seq_length)

Prediction value, pre-rounding: 0.999961
Non-negative review detected.


In [ ]:
test_review_neu = "@AmericanAir i need someone to help me out"
predict(net, test_review_neu, seq_length)

Prediction value, pre-rounding: 0.622515
Non-negative review detected.
